In [1]:
%pylab inline

In [2]:
import sys, os
from IPython.html.widgets import interact
from IPython.display import Image, display
top_dir = '/Users/rjl/student_workshop2'
codes_dir = os.path.join(top_dir, 'Codes')
sys.path.append(codes_dir)

In [3]:
CCmap = imread('/Users/rjl/student_workshop2/maps/CCmap-small.png')
extent = (235.79781, 235.82087, 41.739671,41.762726)   #small region

In [4]:
def plot_CCmap():
    fig = figure(figsize=(6,6))
    ax = axes()
    imshow(CCmap,extent=extent)
    CClatitude = 41.75  # to rescale longitude
    ax.set_aspect(1. / cos(pi*CClatitude/180.)) 
    ax.ticklabel_format(format='plain',useOffset=False)
    return fig

In [5]:
fixed_grid_file = os.path.join(top_dir, 'Datafiles/fixedgrid_xyB_small.npy')
d=load(fixed_grid_file)
x=d[:,0]
y=d[:,1]
B=d[:,2]
topo = reshape(B, (250,250), order='F')
X = reshape(x, (250,250), order='F')
Y = reshape(y, (250,250), order='F')

In [6]:
events_dir = os.path.join(top_dir, 'Events')

In [7]:
zeta1=[0.0,.1,.2,.3,.4,.5,.6,.7,.8,.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8]
zeta2=[1.9,2.0,2.5,3.0,3.5,4.0,4.5,5.0,5.5,6.0]
zeta3=[7.0,8.0,9.0,10.0,11.0,12.0]
zeta=array(zeta1+zeta2+zeta3)
nzeta = len(zeta)
print '%i exceedance values, zeta = \n %s' % (nzeta,zeta)


In [32]:
zeta = hstack((arange(0,2.,.1), arange(2.0,12.5,.5)))
nzeta = len(zeta)
print '%i exceedance values, zeta = \n %s' % (nzeta,zeta)

In [33]:
def combine_prob(p1,p2):
    return 1. - (1-p1)*(1-p2)

In [34]:
prob = zeros((250,250,nzeta))  # initialize to zero

In [35]:
event = 'AASZa'
event_dir = os.path.join(events_dir, event)
hmax_file = os.path.join(event_dir, 'h_eta_small.npy')
hmax = load(hmax_file)
Hmax = hmax.reshape((250,250),order='F')
p_event = 0.1

In [36]:
for k in range(nzeta):
    Pk = prob[:,:,k]  # probabilities at all points for one exceedance value
    prob[:,:,k] = where(Hmax > zeta[k], combine_prob(p_event,Pk), Pk)

In [41]:
prob_clines = [1e-5, 1e-4, 1e-3, 2e-3, 1e-2, 2e-2, 1e-1]
nlines = len(prob_clines)
n1 = int(floor((nlines-1)/2.))
n2 = nlines - 1 - n1
Green = hstack([linspace(1,1,n1),linspace(1,0,n2)])
Red = hstack([linspace(0,0.8,n1), ones(n2)])
Blue = hstack([linspace(1,0.2,n1), zeros(n2)])
prob_colors = zip(Red,Green,Blue)

In [44]:
def plot_pmap(k):
    fig = plot_CCmap()
    contourf(X ,Y, prob[:,:,k], prob_clines, colors=prob_colors, alpha = 0.6)
    title("Annual probability of flooding above %g meters" % zeta[k])
    colorbar()
    return fig

In [45]:
interact(plot_pmap, k=(0,nzeta-1,2))

In [15]:
all_events = """AASZa AASZb AASZc AASZd CSZa CSZb CSZc CSZd CSZe CSZf 
    KmSZa KrSZa SChSZa TOHa""".split()
# print events

In [16]:
event_prob = {}
event_prob['AASZa'] = 1./394.
event_prob['AASZb'] = 1./750.
event_prob['AASZc'] = 1./563.
event_prob['AASZd'] = 1./324.
event_prob['CSZa'] = 1./250. * .0125
event_prob['CSZb'] = 1./250. * .0125
event_prob['CSZc'] = 1./250. * .0750
event_prob['CSZd'] = 1./250. * .5000
event_prob['CSZe'] = 1./250. * .1750
event_prob['CSZf'] = 1./250. * .2250
event_prob['KmSZa'] = 1./50.
event_prob['KrSZa'] = 1./167.
event_prob['SChSZa'] = 1./300.
event_prob['TOHa'] = 1./103.

In [17]:
events = all_events
#events = """AASZa AASZb AASZc AASZd""".split()

In [18]:
prob = zeros((250,250,nzeta))  # initialize to zero
for event in events:
    event_dir = os.path.join(events_dir, event)
    hmax_file = os.path.join(event_dir, 'h_eta_small.npy')
    hmax = load(hmax_file)
    Hmax = hmax.reshape((250,250),order='F')
    for k in range(nzeta):
        Pk = prob[:,:,k]  # probabilities at all points for one exceedance value
        prob[:,:,k] = where(Hmax > zeta[k], combine_prob(event_prob[event],Pk), Pk)

In [20]:
def plot_pmap(k):
    fig = plot_CCmap()
    contourf(X,Y,prob[:,:,k],prob_clines,colors=prob_colors,alpha = 0.6)
    title("Annual probability of flooding above %g meters" % zeta[k])
    colorbar()
    return fig

In [21]:
prob.max()

In [22]:
interact(plot_pmap, k=(0,30,2))

In [23]:
dx = X[1,0] - X[0,0]
dy = Y[0,1] - Y[0,0]
nx, ny = X.shape
xmin = X.min(); xmax = X.max()
ymin = Y.min(); ymax = Y.max()

def plot_hcurve(longitude, latitude):
    i = (longitude - X[0,0]) / dx
    j = (latitude - Y[0,0]) / dy
    if (i<0) or (i>=nx) or (j<0) or (j>=ny):
        print "out of domain"
        return 
    fig = figure(figsize=(12,5))
    subplot(1,2,1)
    p = maximum(prob[i,j,:], 1e-10)
    semilogy(zeta, p, 'b')
    ylim(1e-5,1)
    
    ax = subplot(1,2,2)
    imshow(CCmap,extent=extent)
    CClatitude = 41.75  # to rescale longitude
    ax.set_aspect(1. / cos(pi*CClatitude/180.)) 
    ax.ticklabel_format(format='plain',useOffset=False)
    plot([longitude], [latitude], 'ro')
    xlim(xmin,xmax)
    ylim(ymin,ymax)
    return fig

In [24]:
interact(plot_hcurve, longitude=(xmin,xmax,.001),latitude=(ymin,ymax,0.001))

In [25]:
clines = [1e-3] + list(linspace(0.5,4.5,9))
nlines = len(clines)
n1 = int(floor((nlines-1)/2.))
n2 = nlines - 1 - n1
Green = hstack([linspace(1,1,n1),linspace(1,0,n2)])
Red = hstack([linspace(0,0.8,n1), ones(n2)])
Blue = hstack([linspace(1,0.2,n1), zeros(n2)])
colors2 = zip(Red,Green,Blue)

In [26]:
K = prob > 1e-3
K[:,:,0] = True
#z = zeta[K[:,:,:]]
K.shape
depth = zeros(X.shape)
for i in range(nx):
    for j in range(ny):
        depth[i,j] = zeta[K[i,j,:]][-1]

In [27]:
def plot_inundation_map(p):
    K = prob > p
    K[:,:,0] = True
    K.shape
    depth = zeros(X.shape)
    for i in range(nx):
        for j in range(ny):
            depth[i,j] = zeta[K[i,j,:]][-1]
            
    fig = plot_CCmap()
    contourf(X,Y,depth,clines,alpha = 0.6)
    title("Depth of flooding for annual probability %g" % p)
    colorbar()
    return fig

fig = plot_inundation_map(0.005)

In [28]:
interact(plot_inundation_map, p=(0.00025,0.01,0.00025))